## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-27-20-49-01 +0000,startribune,Afghan national charged in Guard ambush shooti...,https://www.startribune.com/afghan-national-in...
1,2025-11-27-20-46-06 +0000,wapo,Afghan national who once worked with the CIA c...,https://www.washingtonpost.com/nation/2025/11/...
2,2025-11-27-20-44-32 +0000,nypost,DC National Guard terror suspect suffered ment...,https://nypost.com/2025/11/27/us-news/dc-natio...
3,2025-11-27-20-39-07 +0000,wapo,Live updates: FBI executing search warrants in...,https://www.washingtonpost.com/dc-md-va/2025/1...
4,2025-11-27-20-39-07 +0000,wapo,Afghan units were valuable allies in a bloody ...,https://www.washingtonpost.com


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2343/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,national,25
3,guard,24
76,trump,24
144,fire,22
141,hong,21


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
13,2025-11-27-20-14-05 +0000,nypost,Prosecutors will seek death penalty against Af...,https://nypost.com/2025/11/27/us-news/prosecut...,116
270,2025-11-26-21-59-46 +0000,nypost,Trump requests 500 more National Guard troops ...,https://nypost.com/2025/11/26/us-news/trump-re...,110
212,2025-11-27-04-06-00 +0000,wsj,Trump Calls D.C. National Guard Shooting ‘Act ...,https://www.wsj.com/us-news/national-guard-sho...,108
0,2025-11-27-20-49-01 +0000,startribune,Afghan national charged in Guard ambush shooti...,https://www.startribune.com/afghan-national-in...,100
118,2025-11-27-14-37-15 +0000,nypost,Wounded National Guard member volunteered to w...,https://nypost.com/2025/11/27/us-news/second-n...,99


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
13,116,2025-11-27-20-14-05 +0000,nypost,Prosecutors will seek death penalty against Af...,https://nypost.com/2025/11/27/us-news/prosecut...
71,94,2025-11-27-17-29-01 +0000,nyt,Hong Kong Fire Shared Similarities With Grenfe...,https://www.nytimes.com/2025/11/26/world/asia/...
66,70,2025-11-27-17-38-05 +0000,nyt,Trump Cut Europe Out of Ukraine Talks. Here’s ...,https://www.nytimes.com/2025/11/27/world/europ...
219,53,2025-11-27-02-53-50 +0000,nypost,"Hunter, Joe Biden pictured together for first ...",https://nypost.com/2025/11/26/us-news/hunter-j...
132,38,2025-11-27-13-17-00 +0000,wsj,"Centrist dealmaker Sen. Jeanne Shaheen (D., N....",https://www.wsj.com/politics/policy/jeanne-sha...
0,35,2025-11-27-20-49-01 +0000,startribune,Afghan national charged in Guard ambush shooti...,https://www.startribune.com/afghan-national-in...
154,33,2025-11-27-11-21-00 +0000,wsj,"Gilt Yields Rise, Sterling Falls as Concerns O...",https://www.wsj.com/economy/central-banking/gi...
106,33,2025-11-27-15-07-44 +0000,bbc,General named new Guinea-Bissau leader a day a...,https://www.bbc.com/news/articles/cvg164nvj77o...
111,32,2025-11-27-15-02-00 +0000,wsj,Pope Leo Heads to Turkey and Lebanon for His F...,https://www.wsj.com/world/europe/pope-leo-firs...
88,31,2025-11-27-16-39-15 +0000,nypost,Disturbing bodycam footage shows Iryna Zarutsk...,https://nypost.com/2025/11/27/us-news/iryna-za...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
